In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
data_dir = 'ASL_Dataset/Train'
img_size = 64
input_shape = (img_size, img_size, 3)
target_size =(img_size,img_size)
batch_size = 32
num_classes = 36

In [3]:
data = tf.keras.preprocessing.image_dataset_from_directory(data_dir)

Found 80515 files belonging to 36 classes.


In [4]:
datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        fill_mode = 'nearest',
        validation_split = 0.2)

In [5]:
train_ds = datagen.flow_from_directory(
    directory=data_dir,
    target_size=target_size,
    class_mode ='categorical',
    batch_size=batch_size,
    subset="training",
   )

Found 64412 images belonging to 36 classes.


In [6]:
validation_ds = datagen.flow_from_directory(
    directory=data_dir,
    target_size=target_size,
    class_mode ='categorical',
    batch_size=batch_size,
    subset="validation",
    )

Found 16103 images belonging to 36 classes.


In [7]:
#model_01.h5
model = Sequential([ 
    keras.Input(shape=input_shape),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Dropout(0.2),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Dropout(0.2),
    layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(512, activation="relu"),
    layers.Dense(num_classes, activation="softmax"),
])

In [8]:
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 21, 21, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0

In [ ]:
checkpoint = ModelCheckpoint("TrainedModels/model_01_64x64_b32_2.h5", monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto')

model.fit(train_ds, validation_data= validation_ds, epochs=30, validation_steps=10,
          callbacks=[checkpoint,early])

Epoch 1/30
2013/2013 [==============================] - 1894s 936ms/step - loss: 2.6418 - accuracy: 0.2021 - val_loss: 2.3809 - val_accuracy: 0.2969

Epoch 00001: val_accuracy improved from -inf to 0.29688, saving model to TrainedModels\model_01_64x64_b32_2.h5
Epoch 2/30
 582/2013 [=======>......................] - ETA: 11:18 - loss: 1.8523 - accuracy: 0.4180